# 2024 年度後期『AI プログラミング実習』期末試験

このファイルは 2/4 (火) に実施される 2024 年度後期「AI プログラミング実習」の期末試験の問題です。実施内容は以下の通りです。
- 実施日時：2025 年 2 月 4 日（火）　9:30 ～ 12:00［1 組］、13:30 ～ 16:00［2 組］（途中退出可能）
- 実施場所：30519, 30520 教室
- 提出方法：該当の Google Classroom にて
- ファイル名：学籍番号_氏名_ML_Final-Exam.ipynb
- 回答方法：各セルの「#--- 回答欄（↓） ---#」から「#--- 回答欄（↑） ---#」の間にプログラムを記述する。また、解答欄以外の部分を修正してはならない。
- 注意事項：インターネットに接続し、調べながら回答することは構いませんが、実施時間中にメールや SNS などを通じて他者とコミュニケーションすることはカンニングとみなし、退場となります。

なお、このプログラムを実行し、実行できたところまでで評価します。上のセルを回答せず、下のセルを回答しても採点しないので注意してください。

## ■データセットの内容

この試験で使用するデータセットは、機械学習の学習でよく使用されるタイタニック号の生存者データ (Titanic - Machine Learning from Disaster) をこの試験用にアレンジしたものです。<br>
格納されているデータフィールドは以下の通りです。

### ◎データフィールド
- PassengerId – 乗客識別ユニーク ID
- Survived – 生存フラグ（0: 死亡, 1: 生存）
- Pclass – チケットクラス
- Name – 乗客の名前
- Sex – 性別（male: 男性, female: 女性）
- Age – 年齢
- SibSp – タイタニックに同乗している兄弟 / 配偶者の数
- Parch – タイタニックに同乗している親 / 子供の数
- Ticket – チケット番号
- Fare – 料金
- Cabin – 客室番号
- Embarked – 出港地（タイタニックへ乗った港）

なお、このデータセットの問題はクラス分類に該当します。


## ■説明変数と目的変数

このデータセット内で使用する説明変数と目的変数は以下の通りです。

### ◎説明変数
- Pclass – チケットクラス　＜カテゴリー変数＞
- Sex – 性別（male: 男性, female: 女性）　＜カテゴリー変数＞
- Age – 年齢
- SibSp – タイタニックに同乗している兄弟/配偶者の数
- Parch – タイタニックに同乗している親/子供の数
- Fare – 料金
- Embarked – 出港地（タイタニックへ乗った港）　＜カカテゴリー変数＞

### ◎目的変数
- Survived – 生存フラグ（0: 死亡, 1: 生存）

In [116]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE, RFECV
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

---
## ■問題１

ここでは、生存実績データを読み込み、前処理を行い、モデルを作成をするところまで問います。この問題をすべて回答すると、成績は B となります。

### ◎ Step 1

1. タイタニックの生存実績データ [titanic_survived_data.csv] を読み込む
1. 説明変数と目的変数を選別し、それぞれ別の変数に格納する
    - 説明変数：Pclass, Sex, Age, SibSp, Parch, Fare, Embarked
    - 目的変数：Embarked

ただし、データを読み込む際、カテゴリー変数（Pclass, Sex, Embarked）のデータ型を object として指定しないと、次の［問題２］で正常に動作しなくなるので注意すること。

In [117]:
#------------------------------------------------------------#
# データの読み込み
#------------------------------------------------------------#
#--- 回答欄（↓） ---#
df = pd.read_csv(r"C:\Users\shaha\Desktop\final_exam_AI_produce\titanic_survived_data.csv")                  # 生存実績データの読み込み
x_data = df.iloc[:, [2, 4, 5, 6, 7, 9, 11]]  # 説明変数を格納 (Pclass, Sex, Age, SibSp, Parch, Fare, Embarked)
y_data = df.iloc[:, 1]                # 目的変数を格納
#--- 回答欄（↑） ---#

display(x_data.head())
display(y_data.head())

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

### ◎ Step 2

1. One-hot Encoding を行う
1. One-hot の対象となるカテゴリー変数は以下の通り
    - カテゴリー変数：Pclass, Sex, Embarked
1. データフレームを再構成する

In [118]:
#------------------------------------------------------------#
# One-hot Encoding
#------------------------------------------------------------#
#--- 回答欄（↓） ---#
ohe_target =  ["Pclass", "Sex", "Embarked"]                  # One-hot Encoding の対象となるカラムを指定

encoder = OneHotEncoder(sparse_output=False)

x_data_ohe = encoder.fit_transform(x_data[ohe_target])                          # 出力は One-hot 対象のカラムのみ
ohe_cols = encoder.get_feature_names_out(input_features=ohe_target)             # カラム名を取得
x_data_ohe = pd.concat([x_data.drop(columns=ohe_target), pd.DataFrame(x_data_ohe, columns=ohe_cols)], axis=1)
#--- 回答欄（↑） ---#

display(x_data_ohe.head())
display(x_data_ohe.columns)

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
0,22.0,1,0,7.2500,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
1,38.0,1,0,71.2833,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,26.0,0,0,7.9250,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,35.0,1,0,53.1000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,35.0,0,0,8.0500,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0


Index(['Age', 'SibSp', 'Parch', 'Fare', 'Pclass_1', 'Pclass_2', 'Pclass_3',
       'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S',
       'Embarked_nan'],
      dtype='object')

### ◎ Step 3

1. train_test_split を行う
1. パラメーターは以下の通りとする
    - test_size=0.2
    - random_state=0
1. データフレームを再構成する

なお、目的変数を指定する際は y_data.astype(int) として int 型でデータ型を指定すること

In [119]:
#------------------------------------------------------------#
# train_test_split
#------------------------------------------------------------#
#--- 回答欄（↓） ---#
x_train, x_test, y_train, y_test = train_test_split(
    x_data_ohe, y_data, test_size=0.2, random_state=42, stratify=y_data)                    

x_train =  pd.DataFrame(x_train, columns=x_data_ohe.columns)                 # DataFrame を再構成
x_test = pd.DataFrame(x_test, columns=x_data_ohe.columns)                   # DataFrame を再構成
y_train = pd.DataFrame(y_train, columns=["Survived"])                  # DataFrame を再構成
y_test =  pd.DataFrame(y_test, columns=["Survived"])                  # DataFrame を再構成
#--- 回答欄（↑） ---#

### ◎ Step 4

1. SimpleImputer を使用して欠損値補完を行う
1. パラメーターは以下の通りとする
    - missing_values=np.nan
    - strategy='mean'
1. データフレームを再構成する

なお、目的変数を指定する際は y_data.astype(int) として int 型でデータ型を指定すること

In [120]:
#------------------------------------------------------------#
# 欠損値補完
#------------------------------------------------------------#
#--- 回答欄（↓） ---#
imputer =    SimpleImputer(missing_values=np.nan, strategy="mean")                         

x_train_imp = imputer.fit_transform(x_train)    # SimpleImputer を実行
x_train_imp =  pd.DataFrame(x_train_imp, columns=x_train.columns)                             # DataFrame を再構成

x_test_imp = imputer.fit_transform(x_test)     # SimpleImputer を実行
x_test_imp = pd.DataFrame(x_test_imp, columns=x_test.columns)                                # DataFrame を再構成
#--- 回答欄（↑） ---#

display(x_train_imp.head())
display(x_train_imp.shape)

display(x_test_imp.head())
display(x_test_imp.shape)

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
0,29.807687,0.0,0.0,56.4958,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
1,29.807687,0.0,0.0,0.0000,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,29.807687,0.0,0.0,221.7792,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,18.000000,0.0,1.0,9.3500,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
4,31.000000,1.0,1.0,26.2500,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


(712, 13)

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
0,24.00,2.0,0.0,24.1500,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
1,44.00,0.0,1.0,16.1000,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
2,22.00,0.0,0.0,7.2250,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
3,41.00,2.0,0.0,14.1083,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
4,29.25,1.0,0.0,15.5000,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0


(179, 13)

### ◎ Step 5

1. StandardScaler を使用して標準化を行う
1. パラメーターは以下の通りとする
    - 特になし
1. データフレームを再構成する

In [121]:
#------------------------------------------------------------#
# 標準化
#------------------------------------------------------------#
#--- 回答欄（↓） ---#
scaler = StandardScaler()                                   # StandardScaler を構成

x_train_std = scaler.fit_transform(x_train_imp)     # StandardScaler を実行
x_train_std = pd.DataFrame(x_train_std, columns=x_train_imp.columns)     

x_test_std = scaler.fit_transform(x_test_imp)      # StandardScaler を実行
x_test_std = pd.DataFrame(x_test_std, columns=x_test_imp.columns)                               # DataFrame を再構成
#--- 回答欄（↑） ---#

display(x_train_std.head())
display(x_train_std.shape)

display(x_test_std.head())
display(x_test_std.shape)

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
0,-5.463229e-16,-0.465084,-0.466183,0.513812,-0.562211,-0.516627,0.906075,-0.742427,0.742427,-0.492527,-0.289333,0.616316,-0.053074
1,-5.463229e-16,-0.465084,-0.466183,-0.662563,-0.562211,1.935631,-1.103661,-0.742427,0.742427,-0.492527,-0.289333,0.616316,-0.053074
2,-5.463229e-16,-0.465084,-0.466183,3.955399,1.778691,-0.516627,-1.103661,-0.742427,0.742427,-0.492527,-0.289333,0.616316,-0.053074
3,-9.078708e-01,-0.465084,0.727782,-0.467874,-0.562211,-0.516627,0.906075,1.346933,-1.346933,-0.492527,-0.289333,0.616316,-0.053074
4,9.167471e-02,0.478335,0.727782,-0.115977,-0.562211,1.935631,-1.103661,1.346933,-1.346933,-0.492527,-0.289333,0.616316,-0.053074


(712, 13)

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
0,-0.405638,1.087142,-0.522951,-0.172088,-0.579501,-0.484234,0.888819,-0.718992,0.718992,-0.439697,-0.374336,0.631219,0.0
1,1.139649,-0.514491,0.986859,-0.316645,-0.579501,-0.484234,0.888819,-0.718992,0.718992,-0.439697,-0.374336,0.631219,0.0
2,-0.560166,-0.514491,-0.522951,-0.476017,-0.579501,-0.484234,0.888819,-0.718992,0.718992,2.274294,-0.374336,-1.584236,0.0
3,0.907856,1.087142,-0.522951,-0.352411,-0.579501,-0.484234,0.888819,-0.718992,0.718992,-0.439697,-0.374336,0.631219,0.0
4,0.000000,0.286326,-0.522951,-0.327420,-0.579501,-0.484234,0.888819,1.390837,-1.390837,-0.439697,2.671397,-1.584236,0.0


(179, 13)

### ◎ Step 6

1. LogisticRegression を使用してモデルの作成を行う
1. パラメーターは以下の通りとする
    - random_state=0
1. 作成したモデルを使用して予測を行う
1. f1 Score, Accuracy Score, ROC AUC Score を求め、モデルを評価する

In [122]:
#------------------------------------------------------------#
# モデルの作成 & 予測 & 評価（LogisticRegression）
#------------------------------------------------------------#
#--- 回答欄（↓） ---#
model = LogisticRegression(random_state=0)                        # LogisticRegression クラスのインスタンス化
model.fit(x_train_std, y_train.values.ravel())                  # モデルの学習

y_pred = model.predict(x_test_std)                    # 説明変数のテストデータから予測
#--- 回答欄（↑） ---#

coef_df = pd.DataFrame(model.coef_, columns=x_train_std.columns)
display(f"intercept_ : {model.intercept_}")                     # バイアス項 (w0)
display(coef_df.T)                                              # 回帰係数

print("F1 Score:", f1_score(y_test, y_pred))                    # F1 スコア
print("Accuracy Score:", accuracy_score(y_test, y_pred))        # Accracy スコア
print("ROC AUC Score:", roc_auc_score(y_test, y_pred))          # ROC AUC Score スコア

'intercept_ : [-0.65447403]'

,0
Age,-0.512105
SibSp,-0.260483
Parch,-0.071096
Fare,0.107403
Pclass_1,0.505850
Pclass_2,0.088634
Pclass_3,-0.506928
Sex_female,0.634470
Sex_male,-0.634470
Embarked_C,0.049590


F1 Score: 0.71875
Accuracy Score: 0.7988826815642458
ROC AUC Score: 0.7742424242424242


### ◎ Step 7

1. RandomForestClassifier を使用してモデルの作成を行う
1. パラメーターは以下の通りとする
    - random_state=0
1. 作成したモデルを使用して予測を行う
1. f1 Score, Accuracy Score, ROC AUC Score を求め、モデルを評価する

In [123]:
#------------------------------------------------------------#
# モデルの作成 & 予測 & 評価（RandomForestClassifier）
#------------------------------------------------------------#
#--- 回答欄（↓） ---#
model = RandomForestClassifier(random_state=0)                        # RandomForestRegressor クラスのインスタンス化
model.fit(x_train_std, y_train.values.ravel())                  # モデルの学習

y_pred = model.predict(x_test_std)                        # 説明変数のテストデータから予測
#--- 回答欄（↑） ---#

fi_df = pd.DataFrame([model.feature_importances_], columns=x_train_std.columns)
display(fi_df.T.sort_values(0, ascending=False))                # 特徴量の影響度

print("F1 Score:", f1_score(y_test, y_pred))                    # F1 スコア
print("Accuracy Score:", accuracy_score(y_test, y_pred))        # Accracy スコア
print("ROC AUC Score:", roc_auc_score(y_test, y_pred))          # ROC AUC Score スコア

,0
Fare,0.255465
Age,0.244512
Sex_male,0.155218
Sex_female,0.125083
Pclass_3,0.051916
SibSp,0.044452
Parch,0.037755
Pclass_1,0.033224
Pclass_2,0.016915
Embarked_S,0.015912


F1 Score: 0.6666666666666666
Accuracy Score: 0.776536312849162
ROC AUC Score: 0.7398550724637681


---
## ■問題２

次に、生存予測データを読み込み、前処理を行い、問題１で作成したモデルから予測値を求めます。この問題をすべて回答すると、成績は A となります。

### ◎ Step 1

1. タイタニックの生存予測データ [titanic_target_data.csv] を読み込む
1. 説明変数選別し、変数に格納する
    - 説明変数：Pclass, Sex, Age, SibSp, Parch, Fare, Embarked

ただし、データを読み込む際、カテゴリー変数（Pclass, Sex, Embarked）のデータ型を object として指定しないと正常に動作しなくなるので注意すること。

In [124]:
# 生存予測データの読み込み
#------------------------------------------------------------#
#--- 回答欄（↓） ---#
df = pd.read_csv(
    r"C:\Users\shaha\Desktop\final_exam_AI_produce\titanic_target_data.csv",
    dtype={"Pclass": "object", "Sex": "object", "Embarked": "object"}  # カテゴリー変数を object 型として指定
)  

# 説明変数を格納
x_target = df[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]]
#--- 回答欄（↑） ---#

display(x_target.head())


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.5,0,0,7.8292,Q
1,3,female,47.0,1,0,7.0000,S
2,2,male,62.0,0,0,9.6875,Q
3,3,male,27.0,0,0,8.6625,S
4,3,female,22.0,1,1,12.2875,S


### ◎ Step 2

1. One-hot Encoding を行う
1. One-hot の対象となるカテゴリー変数は以下の通り
    - カテゴリー変数：Pclass, Sex, Embarked
1. データフレームを再構成する

In [125]:
#------------------------------------------------------------#
# One-hot Encoding
#------------------------------------------------------------#
#--- 回答欄（↓） ---#
ohe_target =                    # One-hot Encoding の対象となるカラムを指定

encoder = OneHotEncoder(sparse_output=False)

x_target_ohe = encoder.fit_transform(...)                      # 出力は One-hot 対象のカラムのみ
ohe_cols = encoder.get_feature_names_out(input_features=ohe_target)             # カラム名を取得
x_target_ohe = pd.concat([x_target.drop(columns=ohe_target), pd.DataFrame(x_target_ohe, columns=ohe_cols)], axis=1)
#--- 回答欄（↑） ---#

display(x_target_ohe.head())
display(x_target_ohe.columns)

SyntaxError: invalid syntax (2535190881.py, line 5)

### ◎ Step 3

1. 生存実績データ [x_data_ohe] の内容を出力する
1. 生存予測データ [x_target_ohe] の内容を出力する

In [ ]:
#------------------------------------------------------------#
# 生存実績データ
#------------------------------------------------------------#
print('■生存実績データ')
display(x_data_ohe.head())
display(x_data_ohe.columns)
display(x_data_ohe.shape)

#------------------------------------------------------------#
# 生存予測データ
#------------------------------------------------------------#
print('■生存予測データ')
display(x_target_ohe.head())
display(x_target_ohe.columns)
display(x_target_ohe.shape)

### ◎ Step 4

1. 生存実績データのみの項目を出力する
1. 生存予測データのみの項目を出力する

この時点で、Pclass_1 と Pclass_1.0 の両方が存在する場合は NG です。

In [ ]:
#--- 回答欄（↓） ---#
print('■生存実績データのみの項目')
print(set(...) - set(...))                  # 生存実績データのみの項目を表示
print()
#--- 回答欄（↑） ---#

#--- 回答欄（↓） ---#
print('■生存予測データのみの項目')
print(set(...) - set(...))                  # 生存予測データのみの項目を表示
print()
#--- 回答欄（↑） ---#

### ◎ Step 5

1. 生存実績データのみに存在する項目 => 生存予測データに値 0 として項目を追加
1. 生存予測データのみに存在する項目 => 生存予測データから項目を削除
1. 生存予測データの列の並びを生存実績データと同じにする

In [126]:
#------------------------------------------------------------#
# 生存予測データに値 0 として項目を追加
#------------------------------------------------------------#
#--- 回答欄（↓） ---#
diff_cols = set(...) - set(...)         # 実績データのみに存在するカラム

diff_frame = pd.DataFrame(0, index=x_target_ohe.index, columns=list(diff_cols))     # 値 0 でデータフレームを作成
x_target_ohe = pd.concat([x_target_ohe, diff_frame], axis=1)            # x_target_ohe に追加
#--- 回答欄（↑） ---#

#------------------------------------------------------------#
# 生存予測データから項目を削除
#------------------------------------------------------------#
#--- 回答欄（↓） ---#
diff_cols = set(...) - set(...)         # 審査データのみに存在するカラム

x_target_ohe = x_target_ohe.drop(list(diff_cols), axis=1)               # x_target_ohe から削除
#--- 回答欄（↑） ---#

#------------------------------------------------------------#
# カラム順を揃える
#------------------------------------------------------------#
x_target_ohe = x_target_ohe.reindex(x_data_ohe.columns, axis=1)         # 審査データの列の並びを実績データと同じにする

TypeError: 'ellipsis' object is not iterable

### ◎ Step 6

1. 生存実績データのみの項目を出力する
1. 生存予測データのみの項目を出力する

この時点で、生存実績データの項目と生存予測データの項目の内容に差異がない（何も項目が出力されない）状態ならば OK です。

In [127]:
#--- 回答欄（↓） ---#
print('■生存実績データのみの項目')
print(set(...) - set(...))                  # 生存実績データのみの項目を表示
print()
#--- 回答欄（↑） ---#

#--- 回答欄（↓） ---#
print('■生存予測データのみの項目')
print(set(...) - set(...))                  # 生存予測データのみの項目を表示
print()
#--- 回答欄（↑） ---#

■生存実績データのみの項目


TypeError: 'ellipsis' object is not iterable

### ◎ Step 7

1. SimpleImputer を使用して欠損値補完を行う
1. パラメーターは以下の通りとする
    - missing_values=np.nan
    - strategy='mean'
1. データフレームを再構成する

なお、目的変数を指定する際は y_data.astype(int) として int 型でデータ型を指定すること

In [128]:
#------------------------------------------------------------#
# 欠損値補完
#------------------------------------------------------------#
#--- 回答欄（↓） ---#
imputer =                                   # SimpleImputer を構成

x_target_imp = imputer.fit_transform(...)   # SimpleImputer を実行
x_target_imp =                              # DataFrame を再構成
#--- 回答欄（↑） ---#

display(x_target_imp.head())
display(x_target_imp.shape)

SyntaxError: invalid syntax (3526975328.py, line 5)

### ◎ Step 8

1. StandardScaler を使用して標準化を行う
1. パラメーターは以下の通りとする
    - 特になし
1. データフレームを再構成する

In [129]:
#------------------------------------------------------------#
# 標準化
#------------------------------------------------------------#
#--- 回答欄（↓） ---#
scaler =                                    # StandardScaler を構成

x_target_std = scaler.fit_transform(...)    # StandardScaler を実行
x_target_std =                              # DataFrame を再構成
#--- 回答欄（↑） ---#

display(x_target_std.head())
display(x_target_std.shape)

SyntaxError: invalid syntax (873759491.py, line 5)

### ◎ Step 9

1. 問題１で作成したモデルを使用し、生存予測データのクラス値を予測する
1. 予測した結果を出力する

In [130]:
#------------------------------------------------------------#
# ターゲットデータの予測
#------------------------------------------------------------#
#--- 回答欄（↓） ---#
y_pred =                    # 生存予測データのクラス値を予測
y_pred = pd.DataFrame(..., columns=['Predicted_Value'])
display(y_pred)
#--- 回答欄（↑） ---#

SyntaxError: invalid syntax (3395247570.py, line 5)

---
## ■問題３

最後に、ハイパーパラメーターチューニングを行います。この問題をすべて回答すると、成績は S となります。<br>
なお、この処理は、本来［問題１］のモデルの作成の手順の中で行うべきものです。

### ◎ Step 1

1. Pipeline を使用してパイプラインを構成する
    - PCA
        - 識別子：'pca'
        - パラメーター：random_state=0
    - RandomForestClassifier
        - 識別子：'est'
        - パラメーター：random_state=0
1. 候補となるハイパーパラメーターを記述する
    - PCA
        - n_components：'mle', None, 10
    - RandomForestClassifier
        - n_estimators：50, 100, 200
        - max_depth：None, 10, 20, 30
1. 選出されたハイパラメーターでモデルの作成する
1. 選出されたハイパラメーターの内容を表示する
1. 最善のモデルを使用して予測を行う
1. f1 Score, Accuracy Score, ROC AUC Score を求め、モデルを評価する

In [131]:
#--- 回答欄（↓） ---#
pipe = Pipeline([
    (...),                  # 識別子 'pca' で、PCA を構成
    (...)                   # 識別子 'est' で、RandomForestClassifier を構成
])
#--- 回答欄（↑） ---#

params = {
    '...': ...,             # PCA のハイパーパラメーターの候補（n_components：残す項目の数）
    '...': ...,             # RandomForestClassifier のハイパーパラメーターの候補（n_estimators：決定木の数）
    '...': ...              # RandomForestClassifier のハイパーパラメーターの候補（max_depth：木の深さ）
}

gscv = GridSearchCV(estimator=..., param_grid=..., scoring='roc_auc', cv=3)
gscv.fit(...)
#--- 回答欄（↑） ---#

print("Best Parameters:", gscv.best_params_)
print("Best Cross-Validation Score:", gscv.best_score_)

#--- 回答欄（↓） ---#
best_model = gscv.best_estimator_
y_pred = best_model.predict(...)
#--- 回答欄（↑） ---#

print("F1 Score:", f1_score(y_test, y_pred))                    # F1 スコア
print("Accuracy Score:", accuracy_score(y_test, y_pred))        # Accracy スコア
print("ROC AUC Score:", roc_auc_score(y_test, y_pred))          # ROC AUC Score スコア

InvalidParameterError: The 'estimator' parameter of GridSearchCV must be an object implementing 'fit'. Got Ellipsis instead.